# Retrieval

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### Document Loader

In [2]:
# WebBaseLoader: 웹 페이지 내용 읽어옴(크롤링)
from langchain_community.document_loaders import WebBaseLoader

url = 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8'

loader = WebBaseLoader(url)
documents = loader.load()
documents[0].metadata['title']
print(documents[0].page_content[2000:5000])

USER_AGENT environment variable not set, consider setting it to identify your requests.


과사전.


이 문서는 한국어 위키백과의 정책입니다.이것은 모든 사용자들이 일반적으로 따라야 하는 널리 인정된 기준입니다. 문서의 변경은 총의를 반영해야 합니다.단축백:정책백:지침백:규칙백:방침WP:POLICY
요약: 위키백과의 정책과 지침은 위키백과 공동체가 지켜야 할 규범과 사례를 모아둔 것을 의미합니다. 본 정책은 정책과 지침이 어떻게 만들어지고 유지하며 지켜야 하는 지에 대해 설명합니다.
정책과 지침(목록)
원칙
다섯 원칙
규칙에 얽매이지 마세요

콘텐츠 정책(핵심)
중립적 시각
확인 가능
독자 연구 금지
위키백과에 대한 오해

행동 정책
총의
분쟁 해결
편집 분쟁
삭제 정책
차단 정책
법적 위협 금지
인신 공격 금지
문서의 소유권
계정 이름
문서 훼손

기타 정책 분류
집행 정책
법적 정책
절차 정책
생존 인물의 전기

목록
정책 목록
지침 목록
vte
위키백과의 정책(policy)과 지침(guideline)은 위키미디어 공동체가 모범적인 활동 방식, 분쟁 해결, 문서의 표준 작성 기준 등을 제시하기 위해 개발한 것들입니다. 이들의 최종 목표는 자유롭고 신뢰할 수 있는 백과사전 작성이지요. 그렇다고 해서 편집하기 전에 모든 정책과 지침을 다 읽어야 하는 것은 아닙니다. 모든 정책과 지침은 으레 다섯 원칙으로 요약되니까요.
위키백과에 고정 불변의 법칙은 없지만, 위키백과의 정책과 지침 문서에는 공동체가 지키자고 합의한 원칙과 모범 사례가 있습니다. 정책은 모든 사용자들이 일반적으로 따르는 규범이고, 지침은 보통 특정 상황에서 따르면 좋은 모범 사례를 뜻합니다. 정책과 지침은 항상 이성적으로, 또한 상식대로 적용해야 합니다.
본 정책 문서는 한국어 위키백과 내 정책과 지침 및 관련 문서의 공동체 규범 (생성, 채택 과정, 유지 및 관리, 준수)에 대해 명시합니다. 한국어 위키백과의 정책과 지침은 오직 한국어판 내에서만 적용되며, 타 언어판에는 다르게 적용되거나 적용되지 않는 규범이 있을 수 있습니다.


최상위 정책
 위키백과:위키미디어 정책  문서를 참

In [3]:
!pip install pypdf

In [4]:
# PyPDFLoader: PDF 문서를 로드해서 텍스트 읽어옴
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('./data/The_Adventures_of_Tom_Sawyer.pdf')
documents = loader.load()
print(documents[0].metadata)
print('=' * 100)
print(documents[2].page_content)

{'producer': '3-Heights(TM) PDF Optimization Shell 5.9.1.5 (http://www.pdf-tools.com)', 'creator': 'Acrobat PDFMaker 7.0 dla programu Word', 'creationdate': '2006-08-26T00:50:00+02:00', 'author': 'GOLDEN', 'company': 'c', 'title': 'Microsoft Word - 1', 'moddate': '2021-01-27T15:00:11+01:00', 'source': './data/The_Adventures_of_Tom_Sawyer.pdf', 'total_pages': 35, 'page': 0, 'page_label': '1'}
The Adventures of                 
Tom Sawyer 
 
MARK TWAIN 
Level 1 
 
Retold by Jacqueline Kehl                                                    
Series Editors: Andy Hopkins and Jocelyn Potter


### Embedding Model

In [5]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')

text = 'The quick brown fox jumps over the lazy dog.'

vector = embedding_model.embed_query(text)
print(len(vector))

1536


In [6]:
docs = [document.page_content for document in documents]
vects = embedding_model.embed_documents(docs)

print(len(vects))
print(len(vects[0]))

35
1536


### Vector Store

In [7]:
!pip install faiss-cpu

In [8]:
from langchain_community.vectorstores import FAISS

text_embeddings = list(zip(docs, vects)) # [(text, vector), (text, vector), ...]
vector_store = FAISS.from_embeddings(text_embeddings, embedding_model)
# vector_store = FAISS.from_documents(documents, embedding_model)

In [10]:
# vector db 저장
vector_store.save_local('./db/faiss') 

In [11]:
# 저장된 vector db 로드
vector_db = FAISS.load_local(
    './db/faiss',
    embedding_model,
    allow_dangerous_deserialization=True # 신뢰할 수 있는 pkl파일 역직렬화 적용
)

In [9]:
vector_store.similarity_search("Tom Sawyer", k=3)

[Document(id='17e476a6-6901-4bcc-b32f-01820e1479a7', metadata={}, page_content='Introduction \n \n \nOne Saturday afternoon Tom wanted to have an adventure                    \nbecause he didn’t want to think about Injun Joe. He went \nto Huck and said, “I’m going to  look for treasure. Do you \nwant to come with me?” \n \nTom Sawyer loves adventures. He has a lot of adventures \nat home, at school, and with his friends. He has one \nadventure in a cave. But why is he there? What does he \nsee in the cave? And why is he afraid? \n \nMark Twain (1835-1910) is a famous American writer. \nHis name was Samuel Clemens. Young Samuel lived in \nHannibal, Missouri, a small town on the Mississippi River. \nHe loved the river and he liked watching the big boats                  \non it. \nSamuel loved adventures. He worked on boats on the \nMississippi River for two years. Then he went to Nevada. \nHe looked for treasure, but he didn’t find it. He worked for \na newspaper there. His stories were

### Retriever

In [12]:
from langchain_classic.chains import RetrievalQA
# from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0
)

retriever = vector_store.as_retriever()
print(retriever)

retrieval_qa = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type='stuff'
)

tags=['FAISS', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002382777E540> search_kwargs={}


In [15]:
retrieval_qa.invoke('마을 무덤에 있던 남자를 누가 죽였나요?')

{'query': '마을 무덤에 있던 남자를 누가 죽였나요?', 'result': '인준 조(Injun Joe)가 의사를 칼로 죽였습니다.'}

In [13]:
retrieval_qa.invoke('인디언 조가 누구를 죽였나요?')

{'query': '인디언 조가 누구를 죽였나요?', 'result': '인디언 조는 의사를 죽였습니다.'}

In [14]:
retrieval_qa.invoke('인디언 조가 의사를 어떻게 죽였나요?')

{'query': '인디언 조가 의사를 어떻게 죽였나요?', 'result': '인디언 조는 의사를 칼로 죽였습니다.'}